<a href="https://colab.research.google.com/github/SeongwonTak/RecommenderSystem/blob/main/Recommendation_with_MatrixFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation_with_MatrixFactorization
https://lsjsj92.tistory.com/569?category=853217
을 기반으로 따라 실습해보았다.




## 데이터셋 준비.
영화 관련 데이터를 가져오려 한다.
평점과 제목만 있으면 된다.

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk

In [2]:
movie_data_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RecoExample/movies_metadata.csv')
movie_ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RecoExample/ratings_small.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
movie_data_raw.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movie_data = movie_data_raw[['id', 'title']]
movie_data.rename(columns={"id":"movieId"}, inplace = True)
movie_data

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
...,...,...
45461,439050,Subdue
45462,111109,Century of Birthing
45463,67758,Betrayal
45464,227506,Satan Triumphant


In [ ]:
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movie_ratings.drop('timestamp', axis = 1, inplace=True)

두 데이터를 합치려고 한다.

In [ ]:
movie_ratings.movieId = movie_ratings.movieId.astype('str')
movie_data.movieId = movie_data.movieId.astype('str')
user_movie_rating = pd.merge(movie_ratings, movie_data, on = 'movieId')
user_movie_rating

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
1,4,1371,4.0,Rocky III
2,7,1371,3.0,Rocky III
3,19,1371,4.0,Rocky III
4,21,1371,3.0,Rocky III
...,...,...,...,...
44989,652,129009,4.0,Love Is a Ball
44990,653,2103,3.0,Solaris
44991,659,167,4.0,K-PAX
44992,659,563,3.0,Starship Troopers


In [ ]:
user_movie_pivot = user_movie_rating.pivot_table('rating',
                                                 index='userId',
                                                 columns='title').fillna(0)

In [ ]:
user_movie_pivot.head()

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,13 Tzameti,1408,15 Minutes,16 Blocks,1900,1942: A Love Story,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,2046,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,3-Iron,300,33 Scenes from Life,"4 Months, 3 Weeks and 2 Days",40 Days and 40 Nights,42nd Street,48 Hrs.,5 Card Stud,5 Days of War,...,World Trade Center,Wuthering Heights,X-Men Origins: Wolverine,X: The Unknown,Xuxa in Crystal Moon,Y Tu Mamá También,Yaji and Kita: The Midnight Pilgrims,Yamakasi,Yankee Doodle Dandy,Yella,Yesterday,You Are Not I,You Instead,You Only Live Twice,You're a Big Boy Now,"You, Me and Dupree",Young Adam,Young Black Stallion,Young Frankenstein,Young Mr. Lincoln,Young and Innocent,"Young, Violent, Dangerous",Z,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zatoichi,Zatôichi's Pilgrimage,Zazie dans le métro,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movie_user_pivot = user_movie_pivot.values.T

In [ ]:
user_movie_pivot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## SVD 적용
scikit-learn에는 다행히 이미 SVD가 구현되어 있다. 이를 적용하자.

In [ ]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components = 12)
matrix = SVD.fit_transform(movie_user_pivot)
matrix.shape

(2794, 12)

In [ ]:
corr = np.corrcoef(matrix)

movie_title = user_movie_pivot.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index('Toy Story')
corr_coffey_hands = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]

## Matrix_Factorization을 활용한 사용자 기반 추천 구현

한편으로는 Matrix Factorization을 활용하여 사용자 기반 추천 또한 구현이 가능하다.

SVD 추천을 위해 먼저 user iD별 추천한 영화 데이터를 표로 만들어야 한다. 이는 pivot화로 가능하다.

In [ ]:
# 사용 데이터 재점검
movie_ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [5]:
# Pivot 정의
df_user_movie_ratings = movie_ratings.pivot(
    index = 'userId',
    columns = 'movieId',
    values = 'rating'
).fillna(0)

In [ ]:
df_user_movie_ratings.head()

movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,100226,100272,1003,100304,100306,100326,100365,100383,100390,1004,100450,100487,100498,1005,100517,100527,100553,100556,100581,1006,1007,100714,100745,1008,100843,1009,101,1010,101025,101070,...,99089,991,99106,99112,99114,99117,99145,99149,992,99220,99270,99273,99276,99296,99320,994,99415,99437,99468,99470,99574,996,99609,99615,99669,99675,997,99728,99741,99764,99795,998,99811,99813,99839,99846,999,99912,99917,99992
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SVD를 적용하기 전 user간의 bias를 보정하기 위해 mean-centered를 활용할 것이다.

mean-centered의 적용은 간단하게, user별로 평균 평점을 빼주면 된다.

In [6]:
matrix = np.matrix(df_user_movie_ratings)
user_ratings_mean = np.mean(matrix, axis = 1)
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [ ]:
matrix

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 4., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [4., 0., 0., ..., 0., 0., 0.],
        [5., 0., 0., ..., 0., 0., 0.]])

In [7]:
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,2.494375,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
1,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,3.970770,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,4.970770,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,4.970770,-0.029230,-0.029230,...,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230
2,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,...,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075
3,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,3.902162,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,4.902162,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,...,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838
4,-0.043128,-0.043128,3.956872,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,3.956872

U, Sigma V를 얻기 위해 Scikit-learn에서 쓰는 SVD가 아닌, **scipy의 SVD**를 사용하자.

Remark. Scikit-learn 에서는 U, sigma, V를 얻을 수 없다.

In [8]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [9]:
sigma

array([105.72437051, 110.15522471, 116.29714259, 118.66831462,
       122.82232277, 135.96585855, 138.92664501, 145.53348705,
       160.48685586, 199.93327534, 242.13488251, 462.13504865])

문제는, sigma는 대각행렬 형태로 나와야 하는데 그렇지가 않다. 이를 변형해줘야 한다.

In [10]:
sigma = np.diag(sigma)
sigma

array([[105.72437051,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        , 110.15522471,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        , 116.29714259,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 118.66831462,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        122.82232277,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,

분해가 되었기에, 이제 다시 원본 행렬로 복구한다.
따라서 내적으로 U, sigma, Vt를 순서대로 곱해준 후 맨 마지막에 평균을 다시 더해서 복구한다.

In [11]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [12]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.079686,0.021779,-0.013837,-0.005870,-0.028877,0.032371,0.000715,-0.004428,-0.005219,0.038195,0.016529,0.005170,0.001626,-0.005785,0.012191,0.035154,-0.003522,-0.014232,0.003249,-0.000856,0.051961,0.003574,0.000443,0.026025,0.003044,0.004454,-0.008929,0.009934,0.028403,-0.006952,0.009034,0.020068,0.035193,0.002793,-0.025641,-0.003540,-0.002402,0.024249,-0.003985,-0.016380,...,-0.003824,-0.005378,-0.003989,-0.005605,-0.001955,0.013038,-0.003296,-0.002892,-0.003944,-0.003296,0.019296,-0.005399,-0.001182,-0.006048,-0.004537,0.010478,-0.002767,-0.003824,0.017265,-0.004352,-0.003824,-0.002239,-0.006509,-0.003944,-0.004459,-0.002767,-0.000654,-0.005847,-0.004101,-0.003989,-0.004324,-0.004352,0.010478,-0.004256,-0.003944,-0.005674,0.018157,-0.005575,-0.005297,-0.003766
1,1.428452,1.608841,0.529476,0.168278,0.520809,1.107473,0.529719,0.089376,0.296270,1.970031,1.334344,0.118010,0.074826,0.268758,0.120281,0.537446,1.121948,0.172813,1.024733,0.156633,1.303286,0.764510,0.437088,0.133792,1.007832,0.040924,0.173616,0.192708,0.375512,0.142522,0.731898,1.663473,2.026017,0.223532,0.963040,0.015549,-0.003812,1.865005,0.084174,0.178060,...,-0.004103,0.003787,0.021431,-0.002261,-0.003485,0.024201,-0.005930,-0.011921,-0.007644,-0.005930,0.014927,-0.004646,-0.013242,-0.003568,-0.004771,0.020680,-0.007758,-0.004103,0.009578,-0.002275,-0.004103,-0.009586,-0.022052,-0.007644,-0.020560,-0.007758,-0.015069,0.011748,0.018696,0.021431,0.013227,-0.002275,0.020680,-0.005245,-0.007644,-0.021019,0.031243,-0.000957,-0.000753,0.026901
2,0.977246,0.396971,0.000299,0.027444,0.021287,0.141458,-0.057134,0.031633,-0.012538,0.383576,0.220850,-0.038466,0.019204,-0.045240,-0.027879,0.269027,0.098080,0.059854,0.281547,-0.015141,0.161387,0.099303,0.081652,-0.033176,0.188148,-0.025235,0.015483,-0.033352,0.101483,0.051774,0.099001,0.590749,0.504426,0.031071,0.295001,0.018826,-0.023755,0.436133,0.005341,0.052381,...,-0.001449,0.018165,-0.002340,0.014027,-0.016947,-0.018432,-0.007804,-0.038814,-0.006007,-0.007804,-0.067163,0.020456,-0.033227,0.018295,0.002262,-0.014190,-0.014160,-0.001449,-0.069277,0.004907,-0.001449,-0.020515,0.002123,-0.006007,-0.013010,-0.014160,-0.039582,0.031612,-0.000640,-0.002340,0.002761,0.004907,-0.014190,-0.000251,-0.006007,-0.003189,-0.026916,0.014637,0.013287,-0.005741
3,1.870844,1.169993,0.252202,0.094831,-0.181713,-0.511953,-0.027820,-0.143080,0.013247,1.461694,0.719000,0.198060,0.135096,-0.546576,0.038095,0.086534,0.237819,-0.157608,0.520049,-0.010963,1.096399,-0.199695,-0.195134,0.141320,-0.376868,0.082517,-0.264907,-0.099196,0.696930,-0.173210,-0.092670,1.140086,1.475077,0.061266,-0.313946,0.040786,0.096345,1.610197,0.000953,-0.209323,...,-0.040754,-0.012327,0.050500,-0.038387,-0.044972,0.066084,-0.054264,-0.041599,0.034544,-0.054264,-0.017077,0.025852,-0.108300,-0.013064,-0.036227,0.054681,-0.067773,-0.040754,-0.041989,-0.027245,-0.040754,-0.081282,-0.028956,0.034544,-0.058357,-0.067773,-0.121809,0.065424,0.042471,0.050500,0.026412,-0.027245,0.054681,0.018450,0.034544,-0.035740,0.088889,-0.019365,-0.017113,0.066559
4,1.182777,0.924903,0.075998,0.061505,0.602680,-0.159825,0.339925,0.081534,-0.079666,0.535018,0.909278,-0.009722,-0.017869,-0.119970,0.049103,0.414719,0.557867,-0.033698,0.662183,0.016655,0.138584,0.019828,-0.006155,0.252244,-0.166409,0.009711,0.081686,0.172767,-0.104868,-0.168075,0.285302,-0.221550,1.128031,-0.170608,0.029838,-0.012698,-0.044847,1.551725,-0.018612,-0.142095,...,-0.033218,-0.006703,-0.031301,-0.027164,-0.011842,0.014824,-0.037080,-0.077254,-0.030890,-0.037080,-0.006448,-0.020813,-0.052527,-0.036681,-0.043567,0.00

Data 준비가 완료되었다.

이제 실제 추천 시스템을 만들고자 한다. 다음 기능을 반영한다.
- 사용자가 보지 않은 영화중에서 평점이 높은 것을 추천하자.
- 단, 위에서 사용한 SVD를 기준으로 한다.

In [16]:
def recommend_movies(df_svd_preds, user_id,
                     ori_movies_df, ori_ratings_df, num_recommendations = 5):
  user_row_number = user_id - 1
  sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending = False)
  user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
  user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending = False)
  recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]
  recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
  recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
  return user_history, recommendations

In [ ]:
already_rated, predictions = recommend_movies(df_svd_preds, 330, movie_data, movie_ratings, 10)

In [ ]:
predictions